In [4]:
import pandas as pd
import json


##read in json into dataframe
with open('Sarcasm_Headlines_Dataset_v2.json', encoding = "utf8") as file:
    data = file.readlines()
data = map(lambda x: x.rstrip(), data)
data_json = "[" + ",".join(data) +"]"
    
data_frame = pd.read_json(data_json)


with open('Sarcasm_Headlines_Dataset.json', encoding = "utf8") as file:
    data = file.readlines()
data = map(lambda x: x.rstrip(), data)
data_json = "[" + ",".join(data) +"]"
    
data_frame_other = pd.read_json(data_json)
print(len(data_frame_other))
print(len(data_frame))
data_test_v2 = data_frame['article_link'].values
data_test_v1 = data_frame_other['article_link'].values



duplicates = []
non_duplicates = []
for x in data_test_v1:
    if x in data_test_v2:
        duplicates.append(x)
    else:
        non_duplicates.append(x)
    
print(len(duplicates))
print(len(non_duplicates))

26709
28619
26709
0


In [44]:
#########################
#########################
##DO NOT RUN THIS AGAIN##
#########################
#########################
import newspaper
import json
import nltk
article_content = {}
links = data_frame['article_link'].values
dead_links = []
counter = 0
for link in links:
    article = newspaper.Article(link)
    try:
        article.download()
        article.parse()
        #dont read in articles with no text
        if article.text:
            article_content[counter] = {}
            article_content[counter]['link'] = link
            article_content[counter]['content'] = article.text
            counter += 1
    except newspaper.ArticleException:
        dead_links.append(link)
        
with open('article_content.json', 'w') as fp:
    json.dump(article_content, fp)

In [5]:
import json
with open('article_content.json') as json_file:
    data_article_content = json.load(json_file)
    
print(data_article_content[str(0)]['link'])


https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html


In [20]:
##stemming test

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer

stem_temp = data_frame['headline'].values

stem_new = []
porter = SnowballStemmer('english', ignore_stopwords = True)
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

for x in stem_temp:
    stem_new.append(stemSentence(x))
    
data_frame['headline'] = stem_new

data_frame.head(5)


,is_sarcastic,headline,article_link
0,1,thirtysometh scientist unveil doomsday clock o...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. total nail why congress is fall short...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggi : 9 delici differ recip,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevent liar from get to work,https://local.theonion.com/inclement-weather-p...
4,1,mother come pretti close to use word stream ' ...,https://www.theonion.com/mother-comes-pretty-c...


In [32]:
##test lemm
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
lem = WordNetLemmatizer()
stem_temp = data_frame['headline'].values
stem_new = []
def lemSentence(sentence):
    token_words = word_tokenize(sentence)
    stem_sentence = []
    POS = nltk.pos_tag(token_words)
    for x in range(0,len(token_words)):
        stem_sentence.append(lem.lemmatize(token_words[x], pos = get_wordnet_pos(POS[x][1])))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
for x in stem_temp:
    stem_new.append(lemSentence(x))


In [20]:
data_frame['headlines'] = stem_new
print(stem_new)

['thirtysomething scientist unveil doomsday clock of hair loss ', 'dem rep. totally nails why congress be fall short on gender , racial equality ', 'eat your veggie : 9 deliciously different recipe ', 'inclement weather prevents liar from get to work ', "mother come pretty close to use word 'streaming ' correctly ", 'my white inheritance ', '5 way to file your tax with less stress ', "richard branson 's global-warming donation nearly as much a cost of failed balloon trip ", 'shadow government get too large to meet in marriott conference room b ', 'lot of parent know this scenario ', 'this lesbian be consider a father in indiana ( and an amazing one at that ) ', "amanda peet tell her daughter sex be ' a special hug ' ", 'what to know regard current treatment for ebola ', "chris christie suggest hillary clinton be to blame for boko haram 's kidnapping of hundred of schoolgirl ", 'ford develop new suv that run purely on gasoline ', 'uber ceo travis kalanick step down from trump economic a

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data_frame['headline'], 
    data_frame['is_sarcastic'], 
    random_state = 1
)
print(X_train, y_train)
print(X_train.shape[0])
print(X_test.shape[0])

14440    woman seamlessly transitions from being too hu...
28261    'it's like biggie and tupac all over again,' s...
12684    westboro baptist church not really sure why th...
11849    police sketch artist admits to only drawing pe...
26075    93-year-old grandmother at thanksgiving worrie...
                               ...                        
10955    terrifying musical number imagines a world wit...
17289    nervous pope candidate changes wine into jesus...
5192     how these mayors rise above the stress of thei...
12172    army drives 9/11 mastermind's lawyer to sacrif...
235      woman in commercial doing yoga to narration of...
Name: headline, Length: 21464, dtype: object 14440    1
28261    1
12684    1
11849    1
26075    1
        ..
10955    0
17289    1
5192     0
12172    0
235      1
Name: is_sarcastic, Length: 21464, dtype: int64
21464
7155


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(stop_words = 'english')
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [10]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
predictions = naive_bayes.predict(testing_data)
predictions

array([1, 0, 0, ..., 1, 1, 1])

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Accuracy score:  0.7997204751921733
Recall score:  0.7997204751921733
Precision score:  0.7996792454964032
F1 score:  0.799680701367876


In [6]:
##new dataframe using full text as features 
counter = 0
non_empty_articles_y = []
non_empty_article_content = []
for index, row in data_frame.iterrows():
    if row['article_link'] == data_article_content[str(counter)]['link']:
        non_empty_articles_y.append(row['is_sarcastic'])
        non_empty_article_content.append(data_article_content[str(counter)]['content'])
        counter += 1
        
new_data_frame = pd.DataFrame()
new_data_frame['article_content'] = non_empty_article_content
new_data_frame['labels'] = non_empty_articles_y

In [29]:
new_data_frame.head()

,article_content,labels
0,Vegetables don't have to be boring or relegate...,0
1,"PROVIDENCE, RI—In spite of his best efforts to...",1
2,"PATERSON, NJ—Family sources told reporters Tue...",1
3,To what extent do you own your inheritance?\n\...,0
4,"Even with years of experience, the process of ...",0


In [40]:
import string
def remove_punctuations_from_string(string1):
    string1 = string1.lower() #changing to lower case
    translation_table = dict.fromkeys(map(ord, string.punctuation), ' ') #creating dictionary of punc & None
    string2 = string1.translate(translation_table) #translating string1
    return string2


stem_temp = new_data_frame['article_content'].values

stem_new = []


for x in stem_temp:
    temp = remove_punctuations_from_string(x)
    stem_new.append(lemSentence(temp))
    
new_data_frame['article_content'] = stem_new

new_data_frame.head()

,article_content,labels
0,vegetable do n t have to be boring or relegate...,0
1,providence ri—in spite of his best effort to b...,1
2,paterson nj—family source tell reporter tuesda...,1
3,to what extent do you own your inheritance lif...,0
4,even with year of experience the process of re...,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    new_data_frame['article_content'], 
    new_data_frame['labels'], 
    random_state = 1
)

In [17]:
count_vector = CountVectorizer(stop_words = 'english')
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [14]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
predictions = naive_bayes.predict(testing_data)
predictions

array([0, 0, 0, ..., 1, 0, 1])

In [61]:
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Accuracy score:  0.9141001466582862
Recall score:  0.9141001466582862
Precision score:  0.9139817950931035
F1 score:  0.9139933650683508


In [19]:
from sklearn import svm

svm = svm.SVC()

svm.fit(training_data, y_train)
y_pred = svm.predict(testing_data)


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))


/home/guillaume/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy score:  0.9321181646763043
Recall score:  0.9321181646763043
Precision score:  0.9320458297100551
F1 score:  0.9320389528219852
